In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scispacy
!pip install spacy-transformers
!python -m spacy download en_core_web_trf
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_scibert-0.5.1.tar.gz

In [ ]:
!pip3 install scikit-learn==0.23.2
!pip3 install nltk
!pip3 install sklearn_crfsuite

In [4]:
!git clone https://github.com/davidsbatista/NER-Evaluation.git

Cloning into 'NER-Evaluation'...
remote: Enumerating objects: 234, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 234 (delta 0), reused 2 (delta 0), pack-reused 230
Receiving objects: 100% (234/234), 82.22 KiB | 20.55 MiB/s, done.
Resolving deltas: 100% (117/117), done.


In [6]:
import nltk
import sklearn_crfsuite

from copy import deepcopy
from collections import defaultdict
import pandas as pd

from sklearn_crfsuite.metrics import flat_classification_report

from NER_Evaluation.ner_evaluation.ner_eval import collect_named_entities
from NER_Evaluation.ner_evaluation.ner_eval import compute_metrics
from NER_Evaluation.ner_evaluation.ner_eval import compute_precision_recall_wrapper
from NER_Evaluation.ner_evaluation.ner_eval import namedtuple
from NER_Evaluation.ner_evaluation.ner_eval import compute_precision_recall
from NER_Evaluation.ner_evaluation.ner_eval import Evaluator

# Preprocess test data

In [7]:
test_data_path = "/content/drive/MyDrive/fine-turning-dataset/sciNER/test.json"
df_test = pd.read_json(test_data_path, lines=True)
df_test

,clusters,sentences,ner,relations,doc_key
0,"[[[59, 64], [73, 73]], [[0, 3], [34, 34]], [[5...","[[Recognition, of, proper, nouns, in, Japanese...","[[[0, 3, Task], [2, 3, OtherScientificTerm], [...","[[[0, 3, 19, 20, PART-OF], [2, 3, 5, 6, PART-O...",X96-1059
1,"[[[83, 86], [102, 102], [129, 129]], [[75, 75]...","[[We, propose, to, incorporate, a, priori, geo...","[[[5, 7, OtherScientificTerm], [10, 15, Method...","[[[5, 7, 10, 15, PART-OF], [23, 24, 32, 35, US...",ICCV_2001_47_abs
2,"[[[74, 77], [85, 85]], [[60, 60], [65, 65]]]","[[This, work, proposes, a, new, research, dire...","[[[10, 12, OtherScientificTerm], [10, 16, Task...","[[], [[23, 26, 30, 31, USED-FOR]], [[47, 48, 5...",INTERSPEECH_2013_21_abs
3,"[[[18, 21], [44, 44]], [[46, 46], [49, 49], [8...","[[Listen-Communicate-Show, -LRB-, LCS, -RRB-, ...","[[[0, 3, Task], [9, 13, Task]], [[18, 21, Meth...","[[[0, 3, 9, 13, USED-FOR]], [[23, 25, 18, 21, ...",H01-1049
4,"[[[1, 3], [17, 17], [88, 88]], [[11, 12], [38,...","[[A, domain, independent, model, is, proposed,...","[[[1, 3, Method], [8, 12, Task], [11, 12, Othe...","[[[1, 3, 8, 12, USED-FOR], [14, 14, 11, 12, FE...",C96-1062
...,...,...,...,...,...
95,"[[[7, 11], [38, 39]], [[3, 3], [43, 43], [85, ...","[[We, introduce, a, method, to, accelerate, th...","[[[3, 3, Generic], [7, 11, Task], [17, 18, Met...","[[[3, 3, 7, 11, USED-FOR], [17, 18, 3, 3, USED...",CVPR_2010_30_abs
96,"[[[57, 57], [120, 120]], [[30, 30], [65, 65]]]","[[Background, modeling, is, an, important, com...","[[[0, 1, Task], [8, 9, Task]], [[23, 26, Other...","[[[0, 1, 8, 9, PART-OF]], [], [[33, 35, 30, 30...",CVPR_2004_30_abs
97,[],"[[Information, distillation, aims, to, extract...","[[[0, 1, Task], [15, 24, Material]], [[36, 38,...","[[], [[36, 38, 41, 44, USED-FOR]], [], []]",INTERSPEECH_2007_40_abs
98,"[[[5, 5], [70, 70]]]","[[This, paper, presents, a, novel, representat...","[[[5, 5, Generic], [7, 8, OtherScientificTerm]...","[[[5, 5, 7, 8, USED-FOR], [12, 14, 7, 8, FEATU...",CVPR_2003_30_abs


In [8]:
processed_sentence_list = []
processed_entity_list = []
for index, row in df_test.iterrows():
  abstract = [" ".join(i) for i in row["sentences"]]
  abstract = " ".join(abstract)
  abstract_list = [item for sublist in row["sentences"] for item in sublist]
  
  for num in range(len(row["sentences"])):
    sentence = row["sentences"][num]
    entity_list = row['ner'][num]
    if entity_list is not None and len(entity_list) != 0:
      text = " ".join(sentence)
      updated_entity_list = []
      # entity_list = entity_list.pop()
      for i in entity_list:
        if type(i) == list and len(i) == 3:
          (start, end, label) = i
          phrase = " ".join(abstract_list[start:end+1])
          phrase = phrase
          if phrase in text:
            updated_entity_list.append((phrase, label))
      entity = [tuple(sublist)for sublist in updated_entity_list]
      processed_sentence_list.append(text)

      processed_entity_list.append(entity)

print(len(processed_sentence_list))
print(len(processed_entity_list))

529
529


In [10]:
def find_index_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll-1

In [16]:
Entity = namedtuple("Entity", "e_type start_offset end_offset")

# Generate predictions (extracted concepts)

In [11]:
import spacy
import spacy_transformers

nlp = spacy.load("/content/drive/MyDrive/fine-turning-dataset/sciNER_fineturn/roBERTa/output/model-best")

In [23]:
from tqdm import tqdm

predicted_entities = []
correct_entities = []

for index in tqdm(range(len(processed_sentence_list))):
  sentence = processed_sentence_list[index]

  sub_predicted_entities = []
  sub_correct_entities = []

  # generate predictions
  output = nlp(sentence)
  extracted_entities = [(ent.text, ent.label_) for ent in output.ents] 
  
  # process predicted entity list
  for entity in extracted_entities:
    if entity != None:
      (text, label) = entity
      if text != None and label != None:
        try:
          start_index, end_index = find_index_sub_list(text.split(), sentence.split())
          sub_predicted_entities.append(Entity(label, start_index, end_index))
        except:
          continue
  
  # process true labeled entity list
  for entity in processed_entity_list[index]:
    if entity != None:
      (text, label) = entity
      start_index, end_index = find_index_sub_list(text.split(), sentence.split())
      sub_correct_entities.append(Entity(label, start_index, end_index))
  
  predicted_entities.append(sub_predicted_entities)
  correct_entities.append(sub_correct_entities)


100%|██████████| 529/529 [00:44<00:00, 11.93it/s]


In [25]:
print(len(predicted_entities))
print(len(correct_entities))

529
529


# Evaluation

In [27]:
tags = ['Task', "Method", "Material", "EvaluationMatric", "OtherScientificTerm", "Generic"]

metrics_results = {'correct': 0, 'incorrect': 0, 'partial': 0,
                   'missed': 0, 'spurious': 0, 'possible': 0, 'actual': 0, 'precision': 0, 'recall': 0}

# overall results
results = {'strict': deepcopy(metrics_results),
           'ent_type': deepcopy(metrics_results),
           'partial':deepcopy(metrics_results),
           'exact':deepcopy(metrics_results)
          }

# results aggregated by entity type
evaluation_agg_entities_type = {e: deepcopy(results) for e in tags}

for true_ents, pred_ents in zip(correct_entities, predicted_entities):

  # compute results for one message
  tmp_results, tmp_agg_results = compute_metrics(true_ents, pred_ents, tags)

  # aggregate overall results
  for eval_schema in results.keys():
    for metric in metrics_results.keys():
        results[eval_schema][metric] += tmp_results[eval_schema][metric]
  
  # Calculate global precision and recall
  results = compute_precision_recall_wrapper(results)

In [28]:
results

{'ent_type': {'correct': 1136,
  'incorrect': 249,
  'partial': 0,
  'missed': 250,
  'spurious': 225,
  'possible': 1635,
  'actual': 1610,
  'precision': 0.7055900621118012,
  'recall': 0.6948012232415902},
 'partial': {'correct': 1225,
  'incorrect': 0,
  'partial': 160,
  'missed': 250,
  'spurious': 225,
  'possible': 1635,
  'actual': 1610,
  'precision': 0.8105590062111802,
  'recall': 0.7981651376146789},
 'strict': {'correct': 1027,
  'incorrect': 358,
  'partial': 0,
  'missed': 250,
  'spurious': 225,
  'possible': 1635,
  'actual': 1610,
  'precision': 0.6378881987577639,
  'recall': 0.6281345565749236},
 'exact': {'correct': 1225,
  'incorrect': 160,
  'partial': 0,
  'missed': 250,
  'spurious': 225,
  'possible': 1635,
  'actual': 1610,
  'precision': 0.7608695652173914,
  'recall': 0.7492354740061162}}